In [22]:
from azureml.core import Workspace, Experiment
from azureml.core.authentication import InteractiveLoginAuthentication

from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

In [23]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [9]:
%%writefile ./score/score.py

# import json
# import numpy as np
# import os
from azureml.core.model import Model
import joblib
from pipe import create_pipeline
# import pandas as pd


def init():
    global model
#     model = 'hallo' 
    model_path = Model.get_model_path('DummyModel')
#     model = model_path
    model = joblib.load(model_path)
    
def run(raw_data):
    return 'haali'

Overwriting ./score/score.py


In [10]:
cd = CondaDependencies.create()
cd.add_pip_package("azureml-defaults")
cd.add_pip_package('pyarrow==0.12.0')
cd.add_pip_package('joblib')
cd.add_pip_package('scikit-learn==0.20.3')
cd.save_to_file(base_directory='./score', conda_file_path='myenv.yml')

'myenv.yml'

In [11]:
myenv = Environment.from_conda_specification(name="myenv", file_path="./score/myenv.yml")

In [12]:
inference_config = InferenceConfig(entry_script="score.py", environment=myenv, source_directory='./score')

In [13]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               auth_enabled=False, # this flag generates API keys to secure access
                                               memory_gb=1,                                 
                                               description='Dummy Classifier for Product Prediction')

In [14]:
service = Model.deploy(workspace=ws, 
                           name='dummy-product-prediction-aci2', 
                           models= [ws.models['DummyModel']], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

In [21]:
print(ws.webservices['dummy-product-prediction-aci2'].get_logs())

2020-06-16T16:59:21,915212634+00:00 - gunicorn/run 
2020-06-16T16:59:21,916106840+00:00 - iot-server/run 
2020-06-16T16:59:21,927673514+00:00 - rsyslog/run 
2020-06-16T16:59:21,938380682+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_c19cf3552ae7d24af5d3c3e41db2bf7e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c19cf3552ae7d24af5d3c3e41db2bf7e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c19cf3552ae7d24af5d3c3e41db2bf7e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c19cf3552ae7d24af5d3c3e41db2bf7e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c19cf3552ae7d24af5d3c3e41db2bf7e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [25]:
ws.webservices['dummy-product-prediction-aci'].get_logs()

ERROR - Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Tue, 16 Jun 2020 17:34:37 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Request-Context': 'appId=cid-v1:6a27ce65-5555-41a3-85f7-b7a1ce31fd6b', 'x-ms-client-request-id': 'e4b9d82628b44d91afdc4988f27430b9', 'x-ms-client-session-id': 'e57ab2fc-0468-41e7-af64-72ce2b556b86', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"ContainerLogNotAvailable","message":"Log of container \'dummy-product-prediction-aci\' in container group \'dummy-product-prediction-aci\' is not available yet. Please check container \'InstanceView\' for more information or retry later."}],"correlation":{"Re

WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Tue, 16 Jun 2020 17:34:37 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Request-Context': 'appId=cid-v1:6a27ce65-5555-41a3-85f7-b7a1ce31fd6b', 'x-ms-client-request-id': 'e4b9d82628b44d91afdc4988f27430b9', 'x-ms-client-session-id': 'e57ab2fc-0468-41e7-af64-72ce2b556b86', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"ContainerLogNotAvailable","message":"Log of container \'dummy-product-prediction-aci\' in container group \'dummy-product-prediction-aci\' is not available yet. Please check container \'InstanceView\' for more infor